In [ ]:
# ======================================
# 1. Install Required Libraries
# ======================================
!pip install -q transformers accelerate peft datasets bitsandbytes

# ======================================
# 2. Imports
# ======================================
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import PromptTuningConfig, get_peft_model, TaskType
from datasets import load_dataset
import torch


In [ ]:
!pip install -q psutil gputil
import threading, psutil, time
import GPUtil

def monitor_ram(interval=5):
    while True:
        mem = psutil.virtual_memory()
        print(f"[RAM] Used: {mem.used / 1024**3:.2f} GB | "
              f"Available: {mem.available / 1024**3:.2f} GB | "
              f"Usage: {mem.percent:.1f}%")
        time.sleep(interval)

def monitor_gpu(interval=5):
    while True:
        gpus = GPUtil.getGPUs()
        for gpu in gpus:
            print(f"[GPU] {gpu.name} | {gpu.memoryUsed:.0f} MB / {gpu.memoryTotal:.0f} MB "
                  f"({gpu.memoryUtil*100:.1f}%)")
        time.sleep(interval)

# Start background threads
threading.Thread(target=monitor_ram, daemon=True).start()
threading.Thread(target=monitor_gpu, daemon=True).start()

In [ ]:
from huggingface_hub import login
from getpass import getpass

token = getpass("🔑 Enter your Hugging Face token: ")
login(token)

In [ ]:
# ======================================
# 3. Load Mistral 7B in 4-bit (memory efficient)
# ======================================
model_name = "mistralai/Mistral-7B-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)


In [ ]:
# ======================================
# 4. Load & Prepare Dataset
# Format: {"prompt": ..., "response": ...}
# ======================================
from datasets import Dataset

# Load JSONL manually and convert to Dataset
import json

with open("prompt_snort_rule.jsonl", "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

dataset = Dataset.from_list(data)

def preprocess(example):
    full_prompt = f"### Question: {example['prompt']}\n### Answer: {example['response']}"
    tokenized = tokenizer(full_prompt, padding="max_length", truncation=True, max_length=512)
    tokenized["labels"] = tokenized["input_ids"].copy()  # 👈 add this line
    return tokenized

dataset = dataset.map(preprocess)

# Split dataset
train_test = dataset.train_test_split(test_size=0.2, seed=42)
train_data = train_test["train"]
eval_data = train_test["test"]


In [ ]:
# ======================================
# 5. Setup Prompt Tuning
# ======================================
peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init="TEXT",
    prompt_tuning_init_text="### Question: What is the capital of France?\n### Answer: Paris.",
    num_virtual_tokens=20,
    tokenizer_name_or_path=model_name
)

model = get_peft_model(model, peft_config)

# ======================================
# 6. Training Arguments
# ======================================
training_args = TrainingArguments(
    output_dir="./mistral_prompt_tuned",   # Save dir
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    max_steps=1000,                         # <-- Set this
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=1e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    report_to="none",
)

# ======================================
# 7. Trainer & Start Training
# ======================================


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data
)


In [ ]:

trainer.train()


In [ ]:
# Save the tuned adapter
model.save_pretrained("./mistral_prompt_tuned")

In [ ]:
# zip for download
!zip -r mistral_prompt_tuned.zip mistral_prompt_tuned